# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos Semanas 7 y 8 : LDA y LMM audio-a-texto**

* **Nombres y matrículas:**

  * A00378771 - Hiram Garcia Austria
  * A01281536 - Joaquín Díaz Hernández
  * A01796568 - Jesús Antonio López Wayas
  * A01795624 - Victor Hugo Vázquez Herrera

* **Número de Equipo: 36**

* ##### **En cada ejercicio pueden importar los paquetes o librerías que requieran.**

* ##### **En cada ejercicio pueden incluir las celdas y líneas de código que deseen.**

In [17]:
# Importamos OpenAI para usar la API de OpenAI
from openai import OpenAI
import openai # Usamos esta tambien porque sino no carga bien la anterior

import pickle # Usamos pickle para guardar los resultados de la API de OpenAI
import json # Usamos json para guardar los procesamientos de los resultados de la API de OpenAI
import os # Usamos os para manejar rutas y archivos
import re # Usamos re para manejar expresiones regulares
import requests # Usamos requests para hacer peticiones HTTP y descargar archivos

import gensim # Usamos gensim para el procesamiento de lenguaje natural
import gensim.corpora as corpora # Usamos gensim.corpora para manejar el diccionario y el corpus
from gensim.models.coherencemodel import CoherenceModel # Usamos CohcerenceModel para evalaur el resultado de nuestro model

import nltk # Usamos nltk para el procesamiento de lenguaje natural
nltk.download('stopwords') # Descargamos las stopwords de NLTK para el español

from nltk.corpus import stopwords # Importamos las stopwords en español de NLTK

stop_words_es = stopwords.words('spanish') # Definimos las stopwords en español

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hille\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # Usamos la clave de API de OpenAI desde las variables de entorno
)

# **Ejercicio 1:**

* #### **Liga de los audios de las fábulas de Esopo:** https://www.gutenberg.org/ebooks/21144

* #### **Descargar los 10 archivos de audio solicitados: 1, 4, 5, 6, 14, 22, 24, 25, 26, 27.**



In [19]:
# Definimos los números de los archivos de audio que vamos a descargar
audio_numbers = [1, 4, 5, 6, 14, 22, 24, 25, 26, 27]

# Definimos la ruta base para los archivos de audio
path_to_audio_file = "https://www.gutenberg.org/files/21144/mp3"
# Definimos el directorio donde se guardarán los archivos de audio descargados
download_directory = "audios_descargados"

list_of_audio_file_names = [] # Lista para almacenar los nombres de los archivos de audio
# Iteramos sobre los números de audio y formateamos los nombres de los archivos
for number in audio_numbers:
    list_of_audio_file_names.append(f"21144-{number:02d}.mp3")

print("--- Archivos de audio a procesar ---")
# Mostramos los nombres de los archivos de audio que vamos a procesar
for audio_file_name in list_of_audio_file_names:
    print(audio_file_name)

# Verificamos si el directorio de descarga existe, si no, lo creamos
if not os.path.exists(download_directory):
    os.makedirs(download_directory)
    print(f"\nDirectorio '{download_directory}' creado.")

print("\n--- Iniciando descarga de audios ---")
# Definimos un diccionario para almacenar las rutas de los archivos de audio descargados
downloaded_audio_paths = {}

# Iteramos sobre la lista de nombres de archivos de audio y los descargamos
for audio_file_name in list_of_audio_file_names:
    audio_url = f"{path_to_audio_file}/{audio_file_name}"
    local_file_path = os.path.join(download_directory, audio_file_name)

    # Verificamos si el archivo ya existe para evitar descargas duplicadas
    try:
        print(f"Descargando: {audio_url} a {local_file_path}")
        response = requests.get(audio_url, stream=True)
        response.raise_for_status()

        with open(local_file_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Descarga completada: {audio_file_name}")
        downloaded_audio_paths[audio_file_name] = local_file_path
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar {audio_file_name}: {e}")
    except Exception as e:
        print(f"Ocurrió un error inesperado durante la descarga de {audio_file_name}: {e}")

print("\n--- Proceso de descarga de audios finalizado. ---")

--- Archivos de audio a procesar ---
21144-01.mp3
21144-04.mp3
21144-05.mp3
21144-06.mp3
21144-14.mp3
21144-22.mp3
21144-24.mp3
21144-25.mp3
21144-26.mp3
21144-27.mp3

--- Iniciando descarga de audios ---
Descargando: https://www.gutenberg.org/files/21144/mp3/21144-01.mp3 a audios_descargados\21144-01.mp3
Descarga completada: 21144-01.mp3
Descargando: https://www.gutenberg.org/files/21144/mp3/21144-04.mp3 a audios_descargados\21144-04.mp3
Descarga completada: 21144-04.mp3
Descargando: https://www.gutenberg.org/files/21144/mp3/21144-05.mp3 a audios_descargados\21144-05.mp3
Descarga completada: 21144-05.mp3
Descargando: https://www.gutenberg.org/files/21144/mp3/21144-06.mp3 a audios_descargados\21144-06.mp3
Descarga completada: 21144-06.mp3
Descargando: https://www.gutenberg.org/files/21144/mp3/21144-14.mp3 a audios_descargados\21144-14.mp3
Descarga completada: 21144-14.mp3
Descargando: https://www.gutenberg.org/files/21144/mp3/21144-22.mp3 a audios_descargados\21144-22.mp3
Descarga comp

# **Ejercicio 2a:**

* #### **Comenten el por qué del modelo seleccionado para extracción del texto de los audios.**

* #### **Extraer el contenido de los audios en texto.**

* #### **Sugerencia:** pueden extraerlo en un formato de diccionario, clave:valor $→$ {audio01:fabula01, ...}

In [20]:
# Definimos una función para transcribir el audio a texto utilizando la API de OpenAI
def audio2file(audio_path_full):
  "Transcribe un archivo de audio a texto utilizando el modelo Whisper-1 de OpenAI."
  try:
    print(f"Transcribiendo audio: {os.path.basename(audio_path_full)}")
    with open(audio_path_full, "rb") as audio_file:

      transcript = client.audio.transcriptions.create(model="whisper-1", # Usamos el modelo Whisper-1 de OpenAI
                                                      file=audio_file, # Archivo de audio a transcribir
                                                      language="es" # Especificamos el idioma del audio como español
                                                      )
    print(f"Transcripción exitosa para {os.path.basename(audio_path_full)}")
    return transcript.text
  except Exception as e:
    print(f"Error al transcribir {os.path.basename(audio_path_full)}: {e}")
    return None

In [21]:
# Definimos el directorio donde se guardarán las transcripciones en formato .pkl
transcripts_directory = "transcripciones_pkl"

# Verificamos si el directorio de transcripciones existe, si no, lo creamos
if not os.path.exists(transcripts_directory):
    os.makedirs(transcripts_directory)
    print(f"\nDirectorio '{transcripts_directory}' creado.")

print("\n--- Iniciando transcripción y guardado de resultados ---")
# Iteramos sobre los archivos de audio
for audio_path_full in downloaded_audio_paths:
    # Obtenemos la ruta completa del archivo de audio
    transcribed_text = audio2file(f"{download_directory}/{audio_path_full}")

    # Si la transcripción fue exitosa, guardamos el texto en un archivo .pkl
    if transcribed_text:
        # Obtenemos el nombre base del archivo de audio sin la extensión
        base_name = os.path.splitext(os.path.basename(audio_path_full))[0]
        # Definimos el nombre del archivo .pkl para guardar la transcripción
        pkl_file_name = f"{base_name}.pkl"
        pkl_full_path = os.path.join(transcripts_directory, pkl_file_name)
        # Verificamos si el archivo .pkl ya existe y lo renombramos si es necesario
        try:
            # Guardamos la transcripción en un archivo .pkl
            with open(pkl_full_path, 'wb') as f:
                pickle.dump(transcribed_text, f)
            print(f"Transcripción guardada en: {pkl_full_path}")
        except Exception as e:
            print(f"Error al guardar el archivo PKL para {pkl_file_name}: {e}")
    else:
        print(f"No se pudo transcribir {os.path.basename(audio_path_full)}, por lo tanto no se guardará un archivo PKL.")

print("\n--- Proceso de transcripción y guardado finalizado. ---")


--- Iniciando transcripción y guardado de resultados ---
Transcribiendo audio: 21144-01.mp3
Transcripción exitosa para 21144-01.mp3
Transcripción guardada en: transcripciones_pkl\21144-01.pkl
Transcribiendo audio: 21144-04.mp3
Transcripción exitosa para 21144-04.mp3
Transcripción guardada en: transcripciones_pkl\21144-04.pkl
Transcribiendo audio: 21144-05.mp3
Transcripción exitosa para 21144-05.mp3
Transcripción guardada en: transcripciones_pkl\21144-05.pkl
Transcribiendo audio: 21144-06.mp3
Transcripción exitosa para 21144-06.mp3
Transcripción guardada en: transcripciones_pkl\21144-06.pkl
Transcribiendo audio: 21144-14.mp3
Transcripción exitosa para 21144-14.mp3
Transcripción guardada en: transcripciones_pkl\21144-14.pkl
Transcribiendo audio: 21144-22.mp3
Transcripción exitosa para 21144-22.mp3
Transcripción guardada en: transcripciones_pkl\21144-22.pkl
Transcribiendo audio: 21144-24.mp3
Transcripción exitosa para 21144-24.mp3
Transcripción guardada en: transcripciones_pkl\21144-24.p

## Comentario por que seccionamos el modelo whisper-1

El modelo Whisper-1 es un modelo de transcripción de audio a texto desarrollado por OpenAI.

Es conocido por su capacidad para manejar múltiples idiomas y dialectos, así como por su precisión en la transcripción de audio.

Este modelo es especialmente útil para transcribir audios en español, ya que ha sido entrenado con una amplia variedad de datos en diferentes idiomas.

A diferencia de otros modelos mas robustos, Whisper-1 es ligero y rápido, lo que lo hace adecuado para actividades académicas y proyectos de investigación como el nuestro.

Cabe mencionar que Whisper-1 es un modelo de código abierto, lo que permite su uso y adaptación en una amplia gama de aplicaciones.

# **Ejercicio 2b:**

* #### **Eliminar el inicio y final comunes de los textos extraídos de cada fábula.**

* #### **Sugerencia:** Pueden guardar esta información en un archivo tipo JSON, para que al estar probando diferentes opciones en los ejercicios siguientes, puedan recuperar rápidamente la información de cada video/fábula.

In [22]:
# Definimos un diccionario para almacenar las transcripciones unificadas
transcriptions_dict = {}

print("\n--- Cargando transcripciones desde archivos PKL ---")
# Iteramos sobre los archivos .pkl en el directorio de transcripciones
for filename in os.listdir(transcripts_directory):
    # Verificamos si el archivo es un archivo .pkl
    if filename.endswith(".pkl"):
        pkl_full_path = os.path.join(transcripts_directory, filename)
        # Intentamos cargar el archivo .pkl
        try:
            with open(pkl_full_path, 'rb') as f:
                transcribed_text = pickle.load(f)
            # Extraemos el número de audio del nombre del archivo usando una expresión regular
            audio_number = re.search(r'(\d{2})\.pkl$', filename).group(1)
            # Creamos una clave para el diccionario de transcripciones
            key = f"audio{audio_number}"
            # Añadimos la transcripción al diccionario
            transcriptions_dict[key] = transcribed_text
            print(f"Cargado {filename} como {key}")
        except Exception as e:
            print(f"Error al cargar {filename}: {e}")

# Definimos la ruta para guardar el archivo JSON unificado
unified_json_path = "transcripciones_unificadas.json"
# Intentamos guardar el diccionario de transcripciones en un archivo JSON
try:
    with open(unified_json_path, 'w', encoding='utf-8') as f:
        json.dump(transcriptions_dict, f, ensure_ascii=False, indent=4)
    print(f"\n--- Transcripciones unificadas guardadas en: {unified_json_path} ---")
except Exception as e:
    print(f"Error al guardar el archivo JSON unificado: {e}")


--- Cargando transcripciones desde archivos PKL ---
Cargado 21144-01.pkl como audio01
Cargado 21144-04.pkl como audio04
Cargado 21144-05.pkl como audio05
Cargado 21144-06.pkl como audio06
Cargado 21144-14.pkl como audio14
Cargado 21144-22.pkl como audio22
Cargado 21144-24.pkl como audio24
Cargado 21144-25.pkl como audio25
Cargado 21144-26.pkl como audio26
Cargado 21144-27.pkl como audio27

--- Transcripciones unificadas guardadas en: transcripciones_unificadas.json ---


In [23]:
# Ahora procesamos el archivo JSON unificado para eliminar inicios y finales comunes
try:
    with open(unified_json_path, 'r', encoding='utf-8') as f:
        unified_transcriptions = json.load(f)
except Exception as e:
    print(f"Error al cargar el archivo JSON para procesamiento: {e}")
    unified_transcriptions = {}
# Definimos un diccionario para almacenar las transcripciones procesadas
processed_transcriptions = {}

# Definimos patrones de inicio y final comunes para eliminar
common_start_pattern = r".+\d+."
common_end_pattern = "Fin de .+"

print("\n--- Procesando transcripciones para eliminar inicios y finales comunes (flexible) ---")
# Iteramos sobre las transcripciones unificadas y aplicamos los patrones de limpieza
for key, text in unified_transcriptions.items():
    # Limpiamos el texto eliminando los inicios y finales comunes
    cleaned_text = text
    cleaned_text = re.sub(common_start_pattern, "", cleaned_text, 1, re.IGNORECASE | re.DOTALL).strip()
    match_end = re.search(common_end_pattern, cleaned_text, re.IGNORECASE | re.DOTALL)
    # Si encontramos un patrón de final común, lo eliminamos
    if match_end:
        cleaned_text = cleaned_text[:match_end.start()].strip()
        print(f"'{key}': Eliminado inicio y final comunes.")
    else:
        print(f"'{key}': Solo eliminado inicio común (o patrón final no encontrado).")
    # Añadimos la transcripción procesada al diccionario
    processed_transcriptions[key] = cleaned_text

# Definimos la ruta para guardar el archivo JSON de transcripciones procesadas
processed_json_path = "transcripciones_procesadas.json"
# Intentamos guardar las transcripciones procesadas en un archivo JSON
try:
    with open(processed_json_path, 'w', encoding='utf-8') as f:
        json.dump(processed_transcriptions, f, ensure_ascii=False, indent=4)
    print(f"\n--- Transcripciones procesadas guardadas en: {processed_json_path} ---")
except Exception as e:
    print(f"Error al guardar el archivo JSON de transcripciones procesadas: {e}")



--- Procesando transcripciones para eliminar inicios y finales comunes (flexible) ---
'audio01': Eliminado inicio y final comunes.
'audio04': Eliminado inicio y final comunes.
'audio05': Eliminado inicio y final comunes.
'audio06': Eliminado inicio y final comunes.
'audio14': Eliminado inicio y final comunes.
'audio22': Eliminado inicio y final comunes.
'audio24': Eliminado inicio y final comunes.
'audio25': Eliminado inicio y final comunes.
'audio26': Eliminado inicio y final comunes.
'audio27': Eliminado inicio y final comunes.

--- Transcripciones procesadas guardadas en: transcripciones_procesadas.json ---


In [24]:
# Mostramos las transcripciones procesadas
processed_transcriptions

{'audio01': 'El lobo y el cordero en el templo Dándose cuenta de que era perseguido por un lobo, un pequeño corderito decidió refugiarse en un templo cercano. Lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro, lo inmolaría a su dios. Mejor así, replicó el cordero, prefiero ser víctima para un dios a tener que perecer en tus colmillos. Si sin remedio vamos a ser sacrificados, más nos vale que sea con el mayor honor.',
 'audio04': 'El Lobo y la Cruz. A un lobo que comía un hueso, se le atragantó el hueso en la garganta y corría por todas partes en busca de auxilio. Encontró en su correra a una grulla y le pidió que le salvara de aquella situación y que enseguida le pagaría por ello. Aceptó la grulla e introdujo su cabeza en la boca del lobo, sacando de la garganta el hueso atravesado. Pidió entonces la cancelación de la paga convenida. Oye, amiga, dijo el lobo, ¿no crees que es suficiente paga con haber sacado tu cabeza sana y salva de mi boca? Nunca hagas favores

# **Ejercicio 3:**

* #### **Apliquen el proceso de limpieza que consideren adecuado.**

* #### **Justifiquen los pasos de limpieza utilizados. Tomen en cuenta que el texto extraído de cada fábula es relativamente pequeño.**

* #### **En caso de que decidan no aplicar esta etapa de limpieza, deberán justificarlo.**

In [25]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...
# Definimos la ruta para guardar el archivo JSON de transcripciones limpias
cleaned_json_path = "transcripciones_limpias.json"
# Definimos un diccionario para almacenar las transcripciones limpias
cleaned_transcriptions = {}

print("\n--- Iniciando proceso de limpieza de texto ---")
# Iteramos sobre las transcripciones procesadas y limpiamos el texto
for key, text in processed_transcriptions.items():
    # Convertimos el texto a minúsculas
    text = text.lower()
    # Eliminamos caracteres especiales y puntuación
    text = re.sub(r'[^\w\s]', ' ', text)
    # Eliminamos números
    text = re.sub(r'\b\d+\b', '', text)
    # Eliminamos múltiples espacios en blanco
    text = re.sub(r'\s+', ' ', text).strip()
    # Eliminamos las stopwords en español
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words_es and len(word) > 1]
    text = ' '.join(filtered_words)
    # Añadimos la transcripción limpia al diccionario definido
    cleaned_transcriptions[key] = text
    print(f"Limpieza completada para '{key}'.")
# Intentamos guardar las transcripciones limpias en un archivo JSON
try:
    with open(cleaned_json_path, 'w', encoding='utf-8') as f:
        json.dump(cleaned_transcriptions, f, ensure_ascii=False, indent=4)
    print(f"\n--- Transcripciones limpias guardadas en: {cleaned_json_path} ---")
except Exception as e:
    print(f"Error al guardar el archivo JSON de transcripciones limpias: {e}")

print("\n¡Proceso de limpieza de texto finalizado!")


--- Iniciando proceso de limpieza de texto ---
Limpieza completada para 'audio01'.
Limpieza completada para 'audio04'.
Limpieza completada para 'audio05'.
Limpieza completada para 'audio06'.
Limpieza completada para 'audio14'.
Limpieza completada para 'audio22'.
Limpieza completada para 'audio24'.
Limpieza completada para 'audio25'.
Limpieza completada para 'audio26'.
Limpieza completada para 'audio27'.

--- Transcripciones limpias guardadas en: transcripciones_limpias.json ---

¡Proceso de limpieza de texto finalizado!


In [26]:
# Mostramos las transcripciones limpias
cleaned_transcriptions

{'audio01': 'lobo cordero templo dándose cuenta perseguido lobo pequeño corderito decidió refugiarse templo cercano llamó lobo dijo si sacrificador encontraba allí adentro inmolaría dios mejor así replicó cordero prefiero ser víctima dios tener perecer colmillos si remedio vamos ser sacrificados vale mayor honor',
 'audio04': 'lobo cruz lobo comía hueso atragantó hueso garganta corría todas partes busca auxilio encontró correra grulla pidió salvara aquella situación enseguida pagaría ello aceptó grulla introdujo cabeza boca lobo sacando garganta hueso atravesado pidió entonces cancelación paga convenida oye amiga dijo lobo crees suficiente paga haber sacado cabeza sana salva boca nunca hagas favores malvados traficantes corruptos pues mucha paga si dejan sano salvo',
 'audio05': 'lobo caballo pasaba lobo sembrado cebada comida gusto dejó siguió camino encontró rato caballo llevó campo comentándole gran cantidad cebada hallado vez comérsela mejor dejado agradaba oír ruido dientes mastic

In [29]:
# Mostramos la longitud de la primera fábula antes y después de ser tratada

# Convertimos a list todos las fábulas para medir el nivel de coherencia de neustro entrenamiento más adelante
cleaned_transcriptions_list = list(cleaned_transcriptions.values())

print(f'Longitud de la Fábula original: {len(list(processed_transcriptions.values())[0])}')
print(f'Longitud de la Fábula después de limpieza: {len(cleaned_transcriptions_list[0])}')

Longitud de la Fábula original: 429
Longitud de la Fábula después de limpieza: 300


### **Justificación de los pasos de limpieza utilizados.**

Es necesario limpiar los textos extraídos de los audios para mejorar la calidad del análisis posterior. Los pasos realizados fueron los siguientes:
1. Convertimos el texto a minusculas para despreocuparnos de mayusculas y minusculas.
2. Eliminamos números y signos de puntuación para enfocarnos en las palabras relevantes.
2. Eliminamos espacios en blanco y extras para evitar problemas de tokenización.
3. Tokenizamos el texto y eliminamos las stopwords (en español) para centrarnos en las palabras clave que aportan significado.
4. Conservamos tokens mayores a 1 carácter para evitar palabras irrelevantes.


# **Ejercicio 4:**

In [36]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

# Definimos una función para extraer palabras clave usando LDA (Latent Dirichlet Allocation)
def extract_lda_keywords(text_content, num_keywords=20):
    """
    Extrae palabras clave de un texto usando LDA, asumiendo un solo tópico.
    """
    # Verificamos si el texto está vacío o solo contiene espacios
    if not text_content.strip():
        return None

    tokens = text_content.split() # Tokenizamos el texto en palabras
    id2word = corpora.Dictionary([tokens]) # Creamos un diccionario de Gensim a partir de los tokens
    corpus = [id2word.doc2bow(tokens)] # Creamos el corpus de Gensim a partir de los tokens

    # Verificamos si el corpus y el diccionario son válidos
    if not corpus or not id2word:
        return None

    # Intentamos crear el modelo LDA y extraer las palabras clave
    try:
        lda_model = gensim.models.ldamodel.LdaModel(
            corpus=corpus,
            id2word=id2word,
            num_topics=1,      # Sirve para extraer uno o varios tópicos, en este caso solo uno
            chunksize=1,       # Procesamos un documento a la vez
            passes=25,         # Número de pasadas para el entrenamiento
            alpha='auto',      # 'auto' puede funcionar mejor que valores fijos
            eta='auto',        # 'auto' puede funcionar mejor que valores fijos
            random_state=100   # Para reproducibilidad
        )
        # Extraemos las palabras clave del modelo LDA
        keywords = lda_model.print_topics(num_topics=1, num_words=num_keywords)

        # Convertimos cleaned_transcriptions_list a lista de listas de tokens
        texts_tokenized = [doc.split() for doc in cleaned_transcriptions_list]

        # Evaluar coherencia
        coherence_model_lda = CoherenceModel(
            model=lda_model,
            texts=texts_tokenized,
            dictionary=id2word,
            coherence='c_v',
            window_size=2,
            topn=10,
        )
        coherence_lda = coherence_model_lda.get_coherence()

        # Retornamos las palabras clave del primer tópico
        if keywords:
            return keywords[0][1], coherence_lda
        else:
            return None
    except Exception as e:
        print(f"Error durante el modelado LDA para el texto: {e}")
        return None




In [37]:
import re

# Definimos un diccionario para almacenar las palabras clave extraídas de cada fábula
fable_keywords = {}
# Definimos el número de palabras clave a extraer por fábula
num_keywords_per_fable = 20

print(f"\n--- Extrayendo palabras clave con LDA (Top {num_keywords_per_fable} palabras) ---")
# Iteramos sobre las transcripciones limpias y extraemos las palabras clave
for fable_id, fable_text in cleaned_transcriptions.items():
    # Extraemos las palabras clave usando la función definida
    keywords, coherence_lda = extract_lda_keywords(fable_text, num_keywords_per_fable)
    # Si se pudieron extraer palabras clave, las guardamos en el diccionario
    if keywords:
        fable_keywords[fable_id] = keywords
        print(f"Palabras clave para {fable_id}:\n{keywords}\n")
        print('Coherencia LDA: ', coherence_lda)
        print('-' * 100)
    else:
        fable_keywords[fable_id] = "No se pudieron extraer palabras clave o texto insuficiente."
        print(f"Advertencia: No se pudieron extraer palabras clave para {fable_id} (texto posiblemente vacío o insuficiente).\n")

# Definimos la ruta para guardar el archivo JSON de palabras clave
keywords_json_path = "fable_keywords_lda.json"
try:
    with open(keywords_json_path, 'w', encoding='utf-8') as f:
        json.dump(fable_keywords, f, ensure_ascii=False, indent=4)
    print(f"\n--- Palabras clave guardadas en: {keywords_json_path} ---")
except Exception as e:
    print(f"Error al guardar el archivo JSON de palabras clave: {e}")


--- Extrayendo palabras clave con LDA (Top 20 palabras) ---
Palabras clave para audio01:
0.052*"lobo" + 0.039*"templo" + 0.039*"cordero" + 0.039*"si" + 0.039*"ser" + 0.039*"dios" + 0.026*"perecer" + 0.026*"perseguido" + 0.026*"prefiero" + 0.026*"refugiarse" + 0.026*"remedio" + 0.026*"replicó" + 0.026*"sacrificados" + 0.026*"adentro" + 0.026*"tener" + 0.026*"vale" + 0.026*"vamos" + 0.026*"mayor" + 0.026*"sacrificador" + 0.026*"pequeño"

Coherencia LDA:  0.5075637440127003
----------------------------------------------------------------------------------------------------
Palabras clave para audio04:
0.043*"lobo" + 0.034*"hueso" + 0.034*"paga" + 0.026*"pidió" + 0.026*"garganta" + 0.026*"boca" + 0.026*"cabeza" + 0.026*"grulla" + 0.017*"todas" + 0.017*"suficiente" + 0.017*"pues" + 0.017*"sacando" + 0.017*"oye" + 0.017*"pagaría" + 0.017*"si" + 0.017*"nunca" + 0.017*"mucha" + 0.017*"malvados" + 0.017*"partes" + 0.017*"sano"

Coherencia LDA:  0.5939447046776785
------------------------------

# **Ejercicio 5a y 5b:**

* #### **5a: Mediante el LLM que hayan seleccionado, generar un único enunciado que describa o resuma cada fábula.**

* #### **5b: Mediante el LLM que hayan seleccionado, generar tres posibles enunciados diferentes relacionados con la historia de la fábula.**

* #### **Sugerencia:** En realidad los dos incisos a y b se pueden obtener con un solo prompt que solicite la información y el formato correspondiente para cada una de estas partes. Por ejemplo, para cada fábula la salida puede ser un primer enunciado genérico que resume o describe dicha temática; seguido de tres enunciados, cada uno hablando sobre una situación o parte diferente de la fábula.

In [38]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

# Definimos las rutas de los archivos JSON que contienen las palabras clave y los resúmenes
keywords_json_path = "fable_keywords_lda.json"
summaries_json_path = "fable_summaries_subtopics.json"

# Verificamos si el archivo de palabras clave existe y lo cargamos
try:
    with open(keywords_json_path, 'r', encoding='utf-8') as f:
        fable_keywords = json.load(f)
    print(f"\n--- Palabras clave cargadas desde: {keywords_json_path} ---")
except FileNotFoundError:
    print(f"Error: El archivo '{keywords_json_path}' no se encontró. Asegúrate de haber ejecutado la etapa de extracción de palabras clave con LDA.")
    exit()
except Exception as e:
    print(f"Error al cargar el archivo JSON de palabras clave: {e}")
    exit()

# Definimos una función para generar un resumen y subtemas utilizando un LLM (Large Language Model)
def generate_summary_and_subtopics(fable_id, keywords_string, model="gpt-4o"):
    "Genera un resumen y tres subtemas para una fábula utilizando un LLM."
    # Verificamos si las palabras clave son válidas
    if not keywords_string or "No se pudieron extraer" in keywords_string:
        print(f"Saltando {fable_id}: No hay palabras clave válidas para procesar.")
        return None

    # Limpiamos la cadena de palabras clave para eliminar números y espacios innecesarios
    keywords_clean = re.sub(r'\*\d+\.\d+', '', keywords_string)
    # Eliminamos espacios adicionales y convertimos a minúsculas
    keywords_list = [kw.strip() for kw in keywords_clean.split('+')]
    # Definimos el prompt para el modelo LLM
    prompt = f"""Basado en las siguientes palabras clave para una fábula en español:
"{', '.join(keywords_list)}"

Por favor, realiza las siguientes tareas:
1. Genera un único enunciado conciso que resuma o describa el tema central de la fábula.
2. Genera tres posibles subtemas (enunciados) diferentes, también concisos, que profundicen o exploren aspectos secundarios de la fábula.

Formato de la respuesta:
Resumen: [Tu enunciado de resumen]
Subtema 1: [Tu primer subtema]
Subtema 2: [Tu segundo subtema]
Subtema 3: [Tu tercer subtema]"""

    # Definimos los mensajes para la API de OpenAI
    messages = [
        {"role": "system", "content": "Eres un experto en fábulas y análisis de texto. Tu objetivo es generar resúmenes y subtemas claros y relevantes basados en palabras clave."},
        {"role": "user", "content": prompt}
    ]

    # Intentamos generar el resumen y los subtemas usando la API de OpenAI
    try:
        print(f"Generando resumen y subtemas para {fable_id} usando {model}...")
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.7, # Controla la creatividad (0.0 es determinista, 1.0 es muy creativo)
            max_tokens=250   # Límite para evitar respuestas excesivamente largas
        )
        content = response.choices[0].message.content.strip()
        print(f"Respuesta recibida para {fable_id}.")

        # Definimos un diccionario para almacenar el resumen y los subtemas
        summary_data = {}
        lines = content.split('\n')
        # Procesamos la respuesta para extraer el resumen y los subtemas
        for line in lines:
            if line.startswith("Resumen:"):
                summary_data["Resumen"] = line.replace("Resumen:", "").strip()
            elif line.startswith("Subtema 1:"):
                summary_data["Subtema 1"] = line.replace("Subtema 1:", "").strip()
            elif line.startswith("Subtema 2:"):
                summary_data["Subtema 2"] = line.replace("Subtema 2:", "").strip()
            elif line.startswith("Subtema 3:"):
                summary_data["Subtema 3"] = line.replace("Subtema 3:", "").strip()

        # Verificamos que se hayan extraído todos los campos necesarios
        if all(k in summary_data for k in ["Resumen", "Subtema 1", "Subtema 2", "Subtema 3"]):
            return summary_data
        else:
            print(f"Advertencia: No se pudieron parsear todos los campos para {fable_id}. Respuesta cruda:\n{content}")
            return {"Resumen": content, "Subtema 1": "N/A", "Subtema 2": "N/A", "Subtema 3": "N/A"}
    except openai.APIConnectionError as e:
        print(f"Error de conexión a la API de OpenAI para {fable_id}: {e}")
        return None
    except openai.RateLimitError as e:
        print(f"Error de límite de tasa (Rate Limit) para {fable_id}: {e}. Espera y reintenta.")
        return None
    except openai.APIStatusError as e:
        print(f"Error de estado de la API para {fable_id}: {e.status_code} - {e.response}")
        return None
    except Exception as e:
        print(f"Ocurrió un error inesperado al llamar a la API de OpenAI para {fable_id}: {e}")
        return None

# Definimos un diccionario para almacenar los resúmenes y subtemas de cada fábula
fable_summaries_and_subtopics = {}

print("\n--- Generando resúmenes y subtemas para cada fábula con LLM ---")
# Iteramos sobre las palabras clave de cada fábula y generamos el resumen y subtemas
for fable_id, keywords_string in fable_keywords.items():
    result = generate_summary_and_subtopics(fable_id, keywords_string, model="gpt-4o")
    if result:
        fable_summaries_and_subtopics[fable_id] = result
    else:
        fable_summaries_and_subtopics[fable_id] = {
            "Resumen": "No se pudo generar el resumen.",
            "Subtema 1": "No se pudo generar el subtema 1.",
            "Subtema 2": "No se pudo generar el subtema 2.",
            "Subtema 3": "No se pudo generar el subtema 3."
        }

try:
    with open(summaries_json_path, 'w', encoding='utf-8') as f:
        json.dump(fable_summaries_and_subtopics, f, ensure_ascii=False, indent=4)
    print(f"\n--- Resúmenes y subtemas guardados en: {summaries_json_path} ---")
except Exception as e:
    print(f"Error al guardar el archivo JSON de resúmenes y subtemas: {e}")

print("\n¡Proceso de generación de resúmenes y subtemas completado!")




--- Palabras clave cargadas desde: fable_keywords_lda.json ---

--- Generando resúmenes y subtemas para cada fábula con LLM ---
Generando resumen y subtemas para audio01 usando gpt-4o...
Respuesta recibida para audio01.
Generando resumen y subtemas para audio04 usando gpt-4o...
Respuesta recibida para audio04.
Generando resumen y subtemas para audio05 usando gpt-4o...
Respuesta recibida para audio05.
Generando resumen y subtemas para audio06 usando gpt-4o...
Respuesta recibida para audio06.
Generando resumen y subtemas para audio14 usando gpt-4o...
Respuesta recibida para audio14.
Generando resumen y subtemas para audio22 usando gpt-4o...
Respuesta recibida para audio22.
Generando resumen y subtemas para audio24 usando gpt-4o...
Respuesta recibida para audio24.
Generando resumen y subtemas para audio25 usando gpt-4o...
Respuesta recibida para audio25.
Generando resumen y subtemas para audio26 usando gpt-4o...
Respuesta recibida para audio26.
Generando resumen y subtemas para audio27 u

In [39]:
# Mostramos los resúmenes y subtemas generados
fable_summaries_and_subtopics

{'audio01': {'Resumen': 'Un cordero, perseguido por un lobo, busca refugio en un templo, prefiriendo enfrentar el sacrificio antes que perecer devorado.',
  'Subtema 1': 'La elección del cordero entre dos peligros, resaltando el dilema de buscar un mal menor como alternativa.',
  'Subtema 2': 'La paradoja de un lugar sagrado que ofrece una solución temporal pero conlleva su propio riesgo.',
  'Subtema 3': 'La crítica a los sacrificios religiosos, cuestionando si el refugio en el templo es realmente una salvación para el cordero.'},
 'audio04': {'Resumen': 'Un lobo, al tragarse un hueso, pide ayuda a una grulla para que lo saque de su garganta, prometiendo pagarle, pero luego se niega a cumplir su promesa.',
  'Subtema 1': 'La grulla arriesga su vida al meter su cabeza en la boca del lobo para ayudarlo, demostrando confianza y valentía.',
  'Subtema 2': 'El lobo muestra ingratitud y malicia al no cumplir su promesa de pagar a la grulla por su ayuda.',
  'Subtema 3': 'La fábula ilustra c

# **Ejercicio 6:**

En esta actividad tuvimos la oportunidad de transformar audios que contenían narraciones de fábulas con diferentes acentos a resúmenes que incluían subtemas de interés para cada una y empleamos el uso de diversas tecnologías del procesamiento de lenguaje natural para poder realizarlo. Tuvimos la oportunidad de emplear una API de OpenAI para la transcripción del audio a texto donde usamos el modelo de whisper-1. Posteriormente lo limpiamos empleando métodos que anteriormente habíamos usado para finalmente usar un modelo de lenguaje como el GPT-4 para generar resúmenes sintetizados. Un paso clave fue la extracción de palabras clave para las fábulas, puesto que usamos el algoritmo de LDA (Latent Dirichlet Allocation), lo que nos permite identificar palabras centrales para cada una.

A su vez, también usamos los mismos procesos con otras fábulas donde tuvimos la oportunidad de tener un contexto más general por la gran diversidad de narraciones, pues en lugar de ser solo diez fueron veinte en total, lo que nos da una idea más clara y concisa de que nuestro procesamiento fue muy bueno, pues los resúmenes y creaciones de subtemas se notan muy buenos.

# **Fin de la actividad LDA y LMM: audio-a-texto**